In [3]:
from transformers import AutoTokenizer, DataCollatorWithPadding
import torch
from Decoder import Decoder
import numpy as np

ModuleNotFoundError: No module named 'Transformers'

Test the Decoder and tensor size with random data

In [ ]:
model = Decoder(20_000, 1024, 16, 64, 4, 2, 0.1)

# 20,000 is the vocab size
x = np.random.randint(0, 20_000, size=(8, 512)) # 20_000 is for easeir reading
x_t = torch.tensor(x)

# Pass the x through the model
y_t = model(x_t) # _t is for tensor
print("y Shapt:", y_t.shape)


In [ ]:
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
from datasets import load_dataset

# we'll use the same dataset, just ignore the labels
raw_datasets = load_dataset("glue", "sst2")

In [ ]:
def tokenize_fn(batch):
  return tokenizer(batch['sentence'], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_fn, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets = tokenized_datasets.remove_columns(
    ["sentence", "idx", "label"])

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=32, # number of samples per batch
    collate_fn=data_collator
)

In [ ]:
# Set autoreload
%reload_ext autoreload
%autoreload 2


import torch
from datetime import datetime

model = Decoder(
    vocab_size=tokenizer.vocab_size,
    max_len=tokenizer.max_model_input_sizes[checkpoint],
    d_k=16,
    d_model=64,
    n_heads=4,
    n_layers=2,
    dropout_prob=0.1,
)

In [ ]:
print ("CUDA:",torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)